## Topic: Comparative Analysis of PM2.5 Levels on Public Holidays, Weekends, and Weekdays in Seoul (by Month)

### Reason for Topic Selection
1. I was curious to see how differences in activity levels between public holidays, weekends, and weekdays influence fine dust concentrations (PM2.5), which led me to choose this topic.
2. Using data from 2023, I aimed to clearly identify monthly patterns and differences by day type through visual comparisons.
3. Initially, I considered analyzing PM2.5 data at the national level, but the differences between day types were not as distinct, which diluted the focus. As a result, I chose Seoul—a high-density, high-activity area—as the subject of this analysis.

### Analysis Summary
This project compares monthly PM2.5 levels in Seoul throughout 2023, visualizing differences across public holidays, weekends, and weekdays.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib
from matplotlib import font_manager

주제: 서울 지역의 월별 공휴일, 주말, 평일의 P2.5 농도 비교 분석
주제 선정 이유
1. 공휴일, 주말, 평일의 활동량 차이가 미세먼지 농도에 어떤 영향을 미치는지 궁금증이 들어서 이 주제를 선택하게 되었습니다. 
2. 2023년 데이터를 활용하여 시각적 비교를 통해 월별 패턴 및 공휴일, 주말, 평일 간의 차이를 명확히 확인할 수 있도록 하였습니다.
3. 초기에는 전국 단위의 미세먼지 농도 변화를 분석하려 했으나, 
  코드 실행 결과 데이터 상에서 평일, 공휴일, 주말의 차이가 명확하지 않아 분석의 초점이 흐려져 인구 밀도가 높고 활동량이 많은 서울을 분석 대상으로 설정하였습니다.
분석 내용: 2023년 데이터를 기반으로 서울 지역 PM2.5 농도를 월별로 비교하고 공휴일과 주말, 평일로 나눠 미세먼지 농도의 분포를 시각화하였습니다.

-----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 한글 폰트를 따로 설정하지 않고 시각화를 진행하였더니 폰트가 깨지는 현상일 발생하여, 
# 이를 해결하기 위해 AppleGothic 폰트를 설정합니다.
try:
    font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"  
    if os.path.exists(font_path):
        font_prop = font_manager.FontProperties(fname=font_path)
        matplotlib.rc('font', family=font_prop.get_name())
        sns.set(style="whitegrid", rc={"font.family": font_prop.get_name()})  
        print(f"설정된 폰트: {font_prop.get_name()}")
    else:
        print("AppleGothic 폰트를 찾을 수 없습니다. 다른 폰트를 시도하세요.")
except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")

In [ ]:
plt.rcParams['axes.unicode_minus'] = False

공휴일 리스트를 정의하였고, 2023년에는 총 15일의 공휴일이 있습니다.
공휴일 데이터를 정의하여 데이터 전처리 및 시각화에 사용합니다.

In [ ]:
holidays = [
    "20230101", "20230121", "20230122", "20230123",
    "20230210", "20230301", "20230505", "20230606",
    "20230815", "20230928", "20230929", "20230930",
    "20231003", "20231009", "20231225"
]

(데이터 로드 및 전처리)
1. 2023년 12개월치 엑셀 파일 데이터를 불러와 하나로 합쳤습니다.(일관성 보장을 위해)
2. 날짜 정보를 활용해 월, 요일, 공휴일, 주말 여부 같은 추가적인 컬럼을 만들었습니다. 
3. 데이터를 깔끔하게 정리해서 분석과 시각화를 바로 진행할 수 있도록 했습니다. 

In [ ]:
def load_and_preprocess_data(file_paths):
    all_data = []  # 모든 데이터를 저장할 리스트를 생성합니다.(각 파일의 데이터를 저장하고 병합하기 위해 리스트 생성)
    start = time.time()
    for file_path in file_paths:
        print(f"Loading file: {file_path}")
        try:
            # 엑셀 데이터 읽어오며, 필요한 열만 로드합니다.
            data = pd.read_excel(file_path, usecols=["지역", "측정일시", "PM25"])
            data.columns = ["지역", "날짜", "PM2.5"] #데이터를 이해하기 쉽도록 컬럼 이름을 변경하였습니다.(PM25->PM2.5)
            # 데이터가 동일한 형식을 유지하기 위해 날짜 포맷을 통일합니다. 
            data["날짜"] = data["날짜"].astype(str).str.zfill(10).str[:8]
            data["월"] = pd.to_datetime(data["날짜"], format="%Y%m%d").dt.month #월 데이터를 추출합니다.
            data["요일"] = pd.to_datetime(data["날짜"], format="%Y%m%d").dt.dayofweek #dayofweek를 사용하여 0~6의 숫자로 요일 추출합니다.(0:월요일, 6:일요일)
            data["공휴일"] = data["날짜"].isin(holidays) # 공휴일 리스트에 포함된 날짜인지 여부를 boolean 값으로 추가합니다.
            data["주말"] = data["요일"].isin([5, 6]) # 요일 값이 5, 6 각각 토요일과 일요일인지 확인해 boolean 값으로 추가합니다.
            
            # 결측값 확인 및 처리합니다.
            missing_values = data.isnull().sum() #isnull().sum()을 사용하여 각 열의 결측값 개수를 계산합니다.
            print(f"결측값 확인 결과: {missing_values}") # 만약 결측값이 존재하면, dropna()로 제거합니다.(결측값으로 인해 발생할 수 있는 오류 방지)
            if missing_values.sum() > 0:
                data = data.dropna()  # 결측값 제거합니다.
            
            # 이상치 확인 및 처리합니다.
            outlier_count = data[(data["PM2.5"] < 0) | (data["PM2.5"] > 500)].shape[0] # PM2.5값이 0미만이거나 500 초가인 데이터는 이상치로 간주합니다.
            print(f"이상치 개수: {outlier_count}") # 데이터 품질을 유지하여 잘못된 결과를 방지하기 위해 PM2.5값이 유효 범위 내에 있는 데이터만 유지하도록 합니다.
            if outlier_count > 0:
                data = data[(data["PM2.5"] >= 0) & (data["PM2.5"] <= 500)]

결측값 및 이상치 확인 결과, 데이터에는 별다른 문제가 발견되지 않았습니다.
--------------------------------------------------------------------------------------------

In [ ]:
#모든 파일의 데이터를 병합하고, 데이터를 로드하고 전처리하는데 걸리는 시간을 측정합니다.
            all_data.append(data)
        except FileNotFoundError as e:
            print(f"File not found: {file_path}")
    combined_data = pd.concat(all_data, ignore_index=True)
    print(f"데이터 로드 및 전처리 완료 (소요 시간: {time.time() - start:.2f}초)")
    return combined_data

(서울 지역 데이터 필터링 함수)
분석 대상인 서울 지역 데이터를 필터링합니다.
이를 통해 서울의 PM2.5 농도만을 분석 대상으로 선정합니다.

In [ ]:
def filter_seoul_data(data):
    start = time.time()
    seoul_data = data[data["지역"].str.contains("서울")]
    print(f"서울 지역 데이터 필터링 완료 (소요 시간: {time.time() - start:.2f}초)")
    return seoul_data

(시각화 함수 월별 공휴일, 주말, 평일 PM2.5 분포 비교)
1. 공휴일, 주말, 평일을 구분하여 데이터에 '구분' 컬럼을 추가합니다.
2. boxplot을 활용해 월별로 구분해 PM2.5 농도의 분포를 시각화합니다.
3. 공휴일, 주말, 평일 간의 차이를 직관적으로 확인할 수 있습니다.
(boxplot을 선택한 이유는 중앙값, 범위, 이상치를 한눈에 볼 수 있어, 월별 PM2.5농도 분포와 특징을 효과적으로 확인할 수 있다고 생각했습니다.)
(또한, 박스플롯의 중앙값이 계절별로 어떻게 변화하는지 확인 가능하며, 공휴일과 평일 간의 차이를 시각적으로 드러납니다.)

In [ ]:
def visualize_monthly_day_type_comparison(data):
    start = time.time()
    data = data.copy()
    data["구분"] = data.apply(
        lambda x: "공휴일" if x["공휴일"] else ("주말" if x["주말"] else "평일"), axis=1
    )
    plt.figure(figsize=(14, 8)) # 그래프 크기를 설정하였습니다.
    sns.boxplot(x="월", y="PM2.5", hue="구분", data=data, palette="Set2") #박스플롯을 생성합니다.
    plt.title("서울 지역 월별 공휴일, 주말, 평일 PM2.5 분포 비교") #그래프 제목을 설정합니다.
    plt.xlabel("월") #X축 레이블입니다.
    plt.ylabel("PM2.5 농도") #Y축 레이블입니다.
    plt.legend(title="구분") #범례 제목입니다.
    plt.grid() # 데이터를 쉽게 읽을 수 있도록 그리드를 추가합니다.
    plt.show()
    print(f"월별 공휴일, 주말, 평일 비교 시각화 완료 (소요 시간: {time.time() - start:.2f}초)")

시각화 후 결과 해석

1. 공휴일, 주말, 평일 분석
그래프에서 공휴일과 주말에는 PM2.5 농도가 평일에 비해 낮은 경향을 확인할 수 있습니다.
공휴일과 주말은 차량 및 공장 가동량 감소로 인해 배출원이 줄어들어 PM2.5 농도가 낮아질 가능성이 있습니다.
설날이나 추석 등 특정 공휴일에는 도시 외곽으로 이동이 증가하면서 도심 지역의 PM2.5농도가 더 낮아질 수 있습니다.

2.달별로(월별) 분석
여름철인 6월~8월의 전반적인 농도가 낮아지는 이유로는 대기 환경 조건이 영향을 미쳤을 가능성이 있습니다.
여름철은 대기 순환이 활발하고 강수량이 많아 PM2.5 농도가 낮아질 가능성이 큽니다.
1~3월에는 특히 서울과 같은 대도시에 난방기구 사용 증가 및 대기 정체 현상으로 인해 미세먼지 농도가 상승할 수 있습니다.
주말에 PM2.5농도가 높게 나오는 이유는 주말에 사람들이 집에 머무는 시간이 길어지고 난방 사용이 집중될 가능성이 있습니다.
3~4월은 황사 현상이 빈번하게 발생하는 시기로, 대기 중 미세먼지 농도가 급증한 것을 그래프를 통해 확인할 수 있습니다.
-------------------------------------------------------------------------------------------

In [ ]:
# 파일 경로 리스트: 이 리스트를 활용해 반복적으로 데이터를 로드하고 처리할 수 있도록 설정합니다.
base_dir = "/Users/bisong/Desktop/빅데이터과제/2023"
file_names = [
    "2023년 1월.xlsx", "2023년 2월.xlsx", "2023년 3월.xlsx",
    "2023년 4월.xlsx", "2023년 5월.xlsx", "2023년 6월.xlsx",
    "2023년 7월.xlsx", "2023년 8월.xlsx", "2023년 9월.xlsx",
    "2023년 10월.xlsx", "2023년 11월.xlsx", "2023년 12월.xlsx"
]
file_paths = [os.path.join(base_dir, file_name) for file_name in file_names]

In [ ]:
# 실행 흐름 확인용 코드입니다.
# 각 단계별로 데이터가 올바르게 처리되고 있는지와 오류 발생시 어디서 문제가 발생하였는지를 확인하기 위해 이를 메시지로 출력할 수 있도록 하였습니다.
try:
    total_start = time.time()
#전체 작업 시간을 알기위해 코드를 작성합니다.
    print("데이터 로드 및 전처리 중이니 기다리시옵소서.")
    data = load_and_preprocess_data(file_paths)
#데이터 로드 및 전처리 과정입니다.
    print("서울 지역 데이터 필터링 중이니 기다리시옵소서.")
    seoul_data = filter_seoul_data(data)
#서울 지역 데이터 필터링하는 과정입니다.
    print("월별 공휴일, 주말, 평일 비교 시각화 중이니 기다리세요.")
    visualize_monthly_day_type_comparison(seoul_data)
#데이터를 시각화하는 과정입니다.

    print(f"전체 작업 완료 (총 소요 시간: {time.time() - total_start:.2f}초)")
except Exception as e:
    print(f"오류 발생: {e}")